<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents

1. [Introduction to scaling Convnets](#1)
2. [Problem formulation](#2)
3. [Scaling dimensions](#3)
4. [Compound scaling method](#4)
5. [EfficientNet architecture](#5)
    1. [Inverted residual block](#5.1)
6. [References](#6)

# Introduction to scaling ConvNets <a name="1"></a>

Since Alexnet won the 2012 ImageNet competition, Convolutional Neural Networks (ConvNets or CNNs) have become ubiquitous in the world of Computer Vision. One of the key issues in designing CNNs, as with all other neural networks, is **model scaling** i.e deciding **how to increase the model size to achieve better accuracy**.

There  are  many  ways  to  scale  a  ConvNet for different resource constraints: [ResNet](https://arxiv.org/abs/1512.03385) can be scaled down (ResNet-18) or up (ResNet-200) by adjusting network depth (number of layers), while [WideResNet](https://arxiv.org/abs/1605.07146) and [MobileNets](https://arxiv.org/abs/1704.04861) can be scaled by network width (number of channels).  It is also known that a bigger input image size will help accuracy with the overhead of more FLOPS (Floating Point Operations Per Second). Although prior studies have shown that network depth and width are both important for ConvNets power, it still remains an open question of how to effectively scale a ConvNet to achieve better efficiency and accuracy.

Google released a paper in 2019 that deal with a new family of CNNs, [**EfficientNet**](https://arxiv.org/abs/1905.11946). These CNNs not only provide better accuracy but also improve the efficiency of the models by reducing the parameters and FLOPS in comparison to the state of art models such as GPipe. 

Their study shows that it is critical to **balance all
dimensions of network width/depth/resolution**, and such balance can be achieved by simply scaling each of them with a constant ratio. Based on this observation, they propose an effective *compound scaling* method.
Unlike conventional practice that arbitrarily scales these factors, their method uniformly scales network width, depth, and resolution with a set of fixed scaling coefficients.  

# Problem Formulation <a name="2"></a>

ConvNet layers are often partitioned into multiple stages or blocks, and all layers in each stage share the same architecture: for example, ResNet has five stages, and all layers in each stage has the same convolutional type (except the first layer which performs down-sampling).

Unlike regular ConvNet designs that mostly focus on finding the best layer architecture $F_i$, model scaling tries to expand the network length ($L_i$), width ($C_i$), and/or resolution $(H_i,W_i)$ without changing $F_i$ predefined in the baseline network. By fixing $F_i$, model scaling simplifies the design problem for new resource constraints, but it still remains a large design space to explore different $L_i$, $C_i$, $H_i$, $W_i$ for each layer. In order to further reduce the design space, authors restrict that all layers must be scaled uniformly with a constant ratio.

# Scaling dimensions <a name="3"></a>

The main difficulty of this problem is that the optimal coefficients for **scaling network width, depth, and resolution depend on each other** and the values change under different resource constraints.  Due to this difficulty, **conventional methods** mostly **scale** ConvNets **in one of these dimensions**:

**Depth**: Scaling network depth is **the most common way** used by many ConvNets.  The intuition is that deeper ConvNet can **capture** richer and **more complex features**, and generalize well on new tasks. However, deeper networks are also **more difficult to train** due to the vanishing gradient problem. Although several techniques, such as skip connections and batch normalization, alleviate the training problem, the accuracy gain of very deep networks diminishes: for example, ResNet-1000 has similar accuracy as ResNet-101 even though it has much more layers. 

**Width**: Scaling network width is **commonly** used **for  small size models**. Wider networks tend to be able to capture more fine-grained features and are **easier to train**. However, extremely wide but shallow networks tend to have **difficulties in capturing higher-level features** (the accuracy quickly saturates when networks become much wider with larger).

**Resolution**: With higher resolution input images, ConvNets can potentially **capture more fine-grained patterns**. Starting from $224 \times 224$ in early ConvNets,  modern ConvNets tend to use $299 \times  299$ or $331 \times 331$ for better accuracy.  Recently, [GPipe](https://arxiv.org/abs/1811.06965) achieves state-of-the-art ImageNet accuracy with $480 \times 480$ resolution. Higher resolutions, such as $600 \times 600$, are also widely used in object detection. Higher resolutions improve accuracy, but the accuracy gain diminishes for very high resolutions.

So, scaling up any dimension of network width, depth, or resolution improves accuracy, but the accuracy gain diminishes for bigger models.

![](https://i.ibb.co/RH3WsYG/scale-up-dim-comvnets.png)

# Compound Scaling method <a name="4"></a>

Authors observed that **different scaling dimensions are not independent**. Intuitively, for higher resolution images, we should increase network depth, such that the larger receptive fields can help capture similar features that include more pixels in bigger images. Correspondingly, we should also increase network width when the resolution is higher, in order to capture more fine-grained patterns with more pixels in high-resolution images. These intuitions suggest that we need to coordinate and balance different scaling dimensions rather than conventional single-dimension scaling.

They concluded that, in order to pursue better accuracy and efficiency, it is critical to **balance all dimensions of network width, depth, and resolution** during ConvNet scaling.

They proposed a new **compound scaling method**, which uses a compound coefficient (specified by the user, that controls how many more resources are available for model scaling) to uniformly scales network width, depth, and resolution in a principled way.

Figure 2 illustrates the difference between the scaling method and conventional methods.

![](https://i.ibb.co/S6nvDbF/model-scaling.png)

# EfficientNet Architecture <a name="5"></a>

Authors evaluated the scaling method using existing ConvNets (MovileNets, ResNets), but in order to better demonstrate the effectiveness of the scaling method, they also developed a new mobile-size baseline, called **EfficientNet**.


Inspired by [MnasNet](https://arxiv.org/abs/1807.11626), they develop a baseline network by leveraging a multi-objective neural architecture search that optimizes both accuracy and FLOPS. Table 1 shows the architecture of EfficientNet-B0. Its main building block is inverted residual block MBConv ([MobileNetV2](https://arxiv.org/abs/1801.04381), [MnasNet](https://arxiv.org/abs/1807.11626)) which we discuss later. They also add [squeeze-and-excitation]((https://arxiv.org/abs/1709.01507)) optimization (SENet) (explained in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)).

![](https://i.ibb.co/wJNXbrH/architecture-efficientnet-b0.png)

Figure 8 compares the ImageNet performance of different scaling methods for the same EfficientNet-B0 baseline network. In general, all scaling methods improve accuracy with the cost of more FLOPS, but the compound scaling method can further improve accuracy, by up to 2.5%, than other single-dimension scaling methods, suggesting the importance of the proposed compound scaling.

![](https://i.ibb.co/MNFn87X/scaling-up-efficiennet.png)

### Inverted Residual Block <a name="5.1"></a>

An **inverted residual block**, sometimes called an MBConv Block, is a **type of residual block** that uses an inverted structure for efficiency reasons. It was originally proposed for the [MobileNetV2](https://arxiv.org/abs/1801.04381) CNN architecture. It has since been reused for several mobile-optimized CNNs.

When we discussed the ResNet architecture (in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)) we saw that deep ResNets use a residual unit called **bottleneck residual block**. It has a `wide -> narrow -> wide` structure concerning the number of channels. That is, the input has a high number of channels, which are compressed with a $1 \times 1$ convolution. Then a $3 \times 3$ convolution. Finally, the number of channels is increased again with a $1 \times 1$ convolution so input and output can be added.

In contrast, an inverted residual block follows a `narrow -> wide -> narrow` approach, hence the inversion. We first widen with a $1 \times 1$ convolution, then we use a $3 \times 3$ **depthwise convolution** (which reduces the number of parameters) (depthwise convolution is explained in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)), and finally we use a $1 \times 1$ convolution to reduce the number of channels so input and output can be added.

The next figure shows the difference between a bottleneck residual block and an inverted residual block:

![](https://i.ibb.co/x86q80k/inverted-residual-block.png)

Diagonally hatched layers do not use non-linearities. The thickness of each block is used to indicate its relative number of channels. Note how **classical residuals connect the layers with a high number of channels, whereas the inverted residuals connect the bottlenecks**.

# References <a name="6"></a>

- [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946)

- [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)

- [Keras: Image classification via fine-tuning with EfficientNet](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

- [Article: EfficientNet: Scaling of Convolutional Neural Networks](https://towardsdatascience.com/efficientnet-scaling-of-convolutional-neural-networks-done-right-3fde32aef8ff)

- [Article: MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://towardsdatascience.com/mobilenetv2-inverted-residuals-and-linear-bottlenecks-8a4362f4ffd5)

- [paperswithcode: residual block](https://paperswithcode.com/method/residual-block)

- [paperswithcode: bottleneck residual block](https://paperswithcode.com/method/bottleneck-residual-block)

- [paperswithcode: inverted residual block](https://paperswithcode.com/method/inverted-residual-block)